<div style="color: rgba(14, 90, 132, 1); vertical-align:middle; text-align: center; font-family: Montserrat;">
    <div style="font-size: 40px; font-weight: bold;">
        Análise Eleições Guarulhos - 1º Turno
    </div>
    <img src="https://solaris-mh8n.onrender.com/static/img/logo_solaris3.png" align="middle" width="30%" height="30%"/>
    <div style="font-size: 20px; font-style: italic; font-family: Montserrat;">
        SOLARIS SISTEMAS <br /> - SS -
    </div>
</div>

In [1]:
import pandas as pd
import warnings
import os
import plotly.graph_objects as go
import folium
import json
import duckdb

from branca.element import Template, MacroElement
from folium.plugins import Draw, BeautifyIcon, MiniMap, MarkerCluster, TimestampedGeoJson, Geocoder
from folium.features import DivIcon
from sqlalchemy import create_engine, text

warnings.filterwarnings("ignore")

In [2]:
path_db = 'D:/db/sdceti03.duckdb'

In [ ]:
q_partido = """
        with v as (SELECT v.sg_partido AS partido, sum(v.qt_votos_nominais_validos) AS total
          FROM tse.voto v 
         WHERE v.cd_municipio = 64777 
           AND v.cd_cargo = 11 
         GROUP BY v.sg_partido)
         select 
         v.partido,
         v.total,
         CASE v.partido 
                WHEN  'PL' THEN 'green' 
        		WHEN  'PT' THEN 'red'
        		WHEN  'PSB' THEN 'yellow'
        		WHEN  'REPUBLICANOS' THEN 'blue'
        		WHEN  'SOLIDARIEDADE' THEN 'orange'
        		WHEN  'PDT' THEN 'white'
        		ELSE 'black'
        	   END AS color 
               from v
               order by v.total desc;
"""

In [ ]:
q_mapa = """
        WITH l AS (SELECT DISTINCT l.nr_zona, l.nm_bairro AS bairro, l.nr_latitude AS lat, l.nr_longitude AS lon 
                     FROM tse.local l 
                    WHERE l.cd_municipio = 64777 
                      AND l.nr_latitude <> -1
                    ORDER BY 1, l.nr_longitude),
             v AS (SELECT v.sg_partido AS partido, v.nr_zona, sum(v.qt_votos_nominais_validos) AS total
                     FROM tse.voto v 
                    WHERE v.cd_municipio = 64777 
                      AND v.cd_cargo = 11 
                    GROUP BY v.sg_partido, v.nr_zona)
        SELECT v.partido, 
               v.total, 
               l.bairro, 
               l.lon, 
               l.lat, 
               l.nr_zona as zona,
               CASE v.partido 
                WHEN  'PL' THEN 'green' 
        		WHEN  'PT' THEN 'red'
        		WHEN  'PSB' THEN 'yellow'
        		WHEN  'REPUBLICANOS' THEN 'blue'
        		WHEN  'SOLIDARIEDADE' THEN 'orange'
        		WHEN  'PDT' THEN 'white'
        		ELSE 'black'
        	   END AS color 
           FROM l INNER JOIN v ON (l.nr_zona = v.nr_zona);
"""

In [ ]:
q_mapa2 = """
WITH l AS (SELECT l.nr_zona AS zona, l.nm_bairro AS bairro, l.nr_latitude AS lat, l.nr_longitude AS lon 
                     FROM tse.local l 
                    WHERE l.cd_municipio = 64777 
                      AND l.nr_latitude <> -1
                    ORDER BY 1, l.nr_longitude),
     l1 AS (SELECT zona, avg(lat) AS lat, avg(lon) AS lon FROM l GROUP BY zona),
     v AS (SELECT v.sg_partido AS partido, v.nr_zona AS zona, sum(v.qt_votos_nominais_validos) AS total
             FROM tse.voto v 
            WHERE v.cd_municipio = 64777 
              AND v.cd_cargo = 11 
            GROUP BY v.sg_partido, v.nr_zona),
     geom_partido AS (SELECT 'PL' AS partido, .006 AS fator UNION ALL
                      SELECT 'PT',            .005 UNION ALL
        		      SELECT 'PSB',           .004 UNION ALL
        		      SELECT 'REPUBLICANOS',  .003 UNION ALL
        		      SELECT 'SOLIDARIEDADE', .002 UNION ALL
        		      SELECT 'PDT',           .001),
     t AS (SELECT v.partido, 
                  v.total,  
                  (SELECT l1.lon FROM l1 WHERE l1.zona = v.zona) AS lon, 
                  (SELECT l1.lat FROM l1 WHERE l1.zona = v.zona) AS lat, 
                  v.zona,
                  CASE v.partido 
                    WHEN 'PL' THEN 'green' 
        		    WHEN 'PT' THEN 'red'
        		    WHEN 'PSB' THEN 'yellow'
        		    WHEN 'REPUBLICANOS' THEN 'blue'
        		    WHEN 'SOLIDARIEDADE' THEN 'orange'
        		    WHEN 'PDT' THEN 'white'
        		ELSE 'black'
        	   END AS color 
           FROM v)
SELECT t.partido, t.total, (t.lon+gp.fator) AS lon, (t.lat+gp.fator) AS lat, t.zona, t.color FROM t INNER JOIN geom_partido gp ON (t.partido = gp.partido);
"""

In [ ]:
con = duckdb.connect(database="D:/db/sdceti03.duckdb", read_only=True)
try:    
    gru = con.sql(q_mapa2).df()
except Exception as e:
    print(e)
finally:
    con.close()
gru['lat'] = gru['lat'].astype(float)
gru['lon'] = gru['lon'].astype(float)
gru.head()

In [ ]:
x = frequency(gru.total.to_list(), [5,15,25,35,50])
x

In [ ]:
gru.total.describe()

In [ ]:
def get_radius(freq):
    if freq < 1000:
        return 5
    elif freq >= 1000 and freq < 10000:
        return 10
    elif freq >= 10000 and freq < 20000:
        return 15
    elif  freq >= 20000 and freq < 30000:
        return 20
    elif freq >= 30000 and freq < 40000:
        return 25
    elif freq >= 40000 and freq < 30000:
        return 35
    else:
        return 50

In [ ]:
gru['radius'] = gru.total.apply(lambda x: get_radius(x))

In [ ]:
gru

In [ ]:
con = duckdb.connect(database="D:/db/sdceti03.duckdb", read_only=True)
try:    
    p = con.sql(q_partido).df()
except Exception as e:
    print(e)
finally:
    con.close()
p.head()

In [ ]:
#pd.crosstab(index=gru['partido'], columns='total')
#gru.head(15)
#gru.info()
print(gru.total.min(), gru.total.max(), (gru.total.max()-gru.total.min()))

In [ ]:
title_html = '<h3 align="center" style="font-size:30px;font-weight: bold;">VOTAÇÃO em Guarulhos</h3>'
mapa = folium.Map(location=[-23.397106352886972, -46.43988504955854], tiles="OpenStreetMap", zoom_start=11, control_scale = True, scrollWheelZoom=True, dragging=True)
mapa.get_root().html.add_child(folium.Element(title_html))

In [ ]:
# Camadas

## Aeroporto
f_aeroporto = 'C:\\py\\sdceti\\Geo\\GeoJson\\aeroporto_limite.geojson'
aeroporto = json.load(open(f_aeroporto))
style1 = {'fillColor': 'yellow', 'color': 'blue', 'weight': 1.0}
folium.GeoJson(aeroporto, style_function=lambda x:style1).add_to(mapa)

## Bairros
f_bairros = 'C:\\py\\sdceti\\Geo\\GeoJson\\bairros_sem_aeroporto.geojson'
bairros = json.load(open(f_bairros, encoding='utf8'))
style2 =  {'color': 'black', 'weight': 2.0, 'fill_opacity': 0.3}
folium.GeoJson(bairros, tooltip=folium.GeoJsonTooltip(fields=['BAIRRO1']), popup=folium.GeoJsonPopup(fields=['BAIRRO1']), style_function=lambda x:style2).add_to(mapa)

In [ ]:
%%time
'''
#tam_bot = 3
#radius=float(data.iloc[i]['value'])*20000,
for i, v in gru.iterrows():
    #icon_circle = BeautifyIcon(icon_shape='circle-dot', border_color=v.color, border_width=v.total)
    folium.Marker(location = [v['lat'], v['lon']], 
                  popup = 'Zona: {}, Partido: {}, Votação: {} '.format(v['zona'], v['partido'], v['total']), 
                  tooltip = 'Zona: {}, Partido: {}, Votação: {} '.format(v['zona'], v['partido'], v['total']), 
                  #icon = icon_circle, 
                  radius=float(v['total'])*20000,
                  fill = True, 
                  fillcolor = v['color'], 
                  color = v['color'], 
                  fill_opacity=1).add_to(mapa)

'''
        
for i,row in gru.iterrows():
    #print(i,row)
    folium.CircleMarker(
      location=[row['lat'], row['lon']],
      radius=row['radius'],
      popup = 'Zona: {}, Partido: {}, Votação: {} '.format(row['zona'], row['partido'], row['total']), 
      tooltip = 'Zona: {}, Partido: {}, Votação: {} '.format(row['zona'], row['partido'], row['total']), 
      #color='crimson',
      color=row['color'],
      fill=True,
      fill_color=row['color']
    ).add_to(mapa)


In [ ]:
template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="pt-br">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>Destaque Industrial em Guarulhos</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

<div id='maplegend' class='maplegend' style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Partidos</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
  """

for i, v in p.iterrows():
    template += "    <li><span style='background:{};'></span>{}-{}</li>".format(v['color'], v['partido'], v['total'])
    
template += """
  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

In [ ]:
mapa.get_root().add_child(macro)

In [ ]:
mapa.save("I:/SDCETI03/INTELIGENCIADADOS/TiozinhoMapero/tse/1_turno_2024.html")

In [ ]:
%%time

cor = 'blue'
tam_bot = 3

for row in gru.itertuples():
    icon_circle = BeautifyIcon(icon_shape='circle-dot', border_color=cor, border_width=tam_bot)
    folium.Marker(location = [row.NR_LATITUDE, row.NR_LONGITUDE], popup = '{}'.format(row.NR_ZONA), tooltip = 'ZONA: {} SEÇÃO: {} BAIRRO: {}'.format(row.NR_ZONA, row.NR_SECAO, row.NM_BAIRRO), icon = icon_circle, fill = True, fillcolor = cor, color = cor, fill_opacity=1).add_to(mapa)
    

In [ ]:
mapa

In [ ]:
voto = pd.read_csv(os.path.join(dir_tse, 'votacao_partido_munzona_2024_SP.csv'), sep=';', encoding='latin1') 

In [ ]:
voto.info()

In [ ]:
voto_gru = voto.query("SG_UF == 'SP' and NM_MUNICIPIO == 'GUARULHOS'")

In [ ]:
voto_gru.info()

In [ ]:
voto_gru.head()

In [ ]:
voto_gru.groupby('DS_CARGO').DS_CARGO.count()

In [ ]:
voto_gru.query("DS_CARGO == 'Prefeito'").iloc[0]

In [ ]:
voto_gru.query("DS_CARGO == 'Prefeito'").groupby('NR_PARTIDO')['QT_VOTOS_NOMINAIS_VALIDOS'].sum()

In [ ]:
%%time

dir_tse = 'D:/dabase_loads/tse_raw'
local = pd.read_csv(os.path.join(dir_tse, 'eleitorado_local_votacao_2024.csv'), sep=';', encoding='latin1') 
local.columns = local.columns.str.lower()
voto = pd.read_csv(os.path.join(dir_tse, 'votacao_partido_munzona_2024_SP.csv'), sep=';', encoding='latin1') 
voto.columns = voto.columns.str.lower()

con = duckdb.connect('d:/db/sdceti03.duckdb')
con.sql('create table tse.local as select * from local')
con.sql('create table tse.voto as select * from voto')

In [ ]:
del con

In [ ]:
dir_tse = 'D:/dabase_loads/tse_raw'
votcan = pd.read_csv(os.path.join(dir_tse, 'votacao_candidato_munzona_2024_SP.csv'), sep=';', encoding='latin1') 
votcan.columns = votcan.columns.str.lower() 
votcan.head()

In [ ]:
con = duckdb.connect('d:/db/sdceti03.duckdb')
con.sql('create table tse.votcan as select * from votcan')
con.close()

In [ ]:
%%time

dir_tse = 'D:/dabase_loads/tse_raw'
local = pd.read_csv(os.path.join(dir_tse, 'detalhe_votacao_secao_2024_SP.csv'), sep=';', encoding='latin1') 
local.columns = local.columns.str.lower()
voto = pd.read_csv(os.path.join(dir_tse, 'votacao_partido_munzona_2024_SP.csv'), sep=';', encoding='latin1') 
voto.columns = voto.columns.str.lower()

con = duckdb.connect('d:/db/sdceti03.duckdb')
con.sql('create table tse.local as select * from local')
con.sql('create table tse.voto as select * from voto')